In [63]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [64]:
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel("Demand Forecast.xlsx")
unit_data = pd.read_excel('Product Data per State.xlsx')
handling_out_data = pd.read_excel('Warehousing.xlsx')

outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

dc_product_demand = {}
for dc in as_is_dc:
    dc_product_demand[dc] = {}  # Create a dictionary for each DC
    for product in ['blender', 'swing', 'chair', 'scooter', 'skiprope']:
        total_demand_per_dc = 0
        for state in as_is_dc[dc]:
            total_demand_per_dc += (demand_data.loc[state, product])
        dc_product_demand[dc][product] = total_demand_per_dc  # Store demand (in boxes) for each product based on DC

dc_product_demand

{'NY': {'blender': 226319,
  'swing': 6468,
  'chair': 161655,
  'scooter': 3233,
  'skiprope': 969935},
 'ND': {'blender': 31949,
  'swing': 913,
  'chair': 22820,
  'scooter': 22832,
  'skiprope': 136921},
 'IL': {'blender': 271812,
  'swing': 7766,
  'chair': 194150,
  'scooter': 3884,
  'skiprope': 1164904},
 'TN': {'blender': 216583,
  'swing': 6188,
  'chair': 154702,
  'scooter': 3095,
  'skiprope': 928213},
 'WA': {'blender': 51437,
  'swing': 1470,
  'chair': 36741,
  'scooter': 735,
  'skiprope': 220442},
 'TX': {'blender': 190768,
  'swing': 5451,
  'chair': 136263,
  'scooter': 2725,
  'skiprope': 817575},
 'UT': {'blender': 179502,
  'swing': 5129,
  'chair': 128215,
  'scooter': 2564,
  'skiprope': 769295}}

In [74]:
warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingIN_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling In', skiprows=1)
warehousing_storageCost = pd.read_excel('Warehousing.xlsx', sheet_name='Storage', skiprows= 1) 

products = ['blender', 'swing', 'chair', 'scooter', 'skiprope']
volume_perBox = [0.0070, 0.0045, 0.0098, 0.0200, 0.0055]

volumes_perStates = {}

for state, products_dict in dc_product_demand.items():
    volumes_perStates[state] = {}
    for i, product in enumerate(products):
        num_boxes = products_dict[product]
        volume = volume_perBox[i]
        volumes_perStates[state][product] = np.ceil(num_boxes * volume)


print(volumes_perStates)

{'NY': {'blender': 1585.0, 'swing': 30.0, 'chair': 1585.0, 'scooter': 65.0, 'skiprope': 5335.0}, 'ND': {'blender': 224.0, 'swing': 5.0, 'chair': 224.0, 'scooter': 457.0, 'skiprope': 754.0}, 'IL': {'blender': 1903.0, 'swing': 35.0, 'chair': 1903.0, 'scooter': 78.0, 'skiprope': 6407.0}, 'TN': {'blender': 1517.0, 'swing': 28.0, 'chair': 1517.0, 'scooter': 62.0, 'skiprope': 5106.0}, 'WA': {'blender': 361.0, 'swing': 7.0, 'chair': 361.0, 'scooter': 15.0, 'skiprope': 1213.0}, 'TX': {'blender': 1336.0, 'swing': 25.0, 'chair': 1336.0, 'scooter': 55.0, 'skiprope': 4497.0}, 'UT': {'blender': 1257.0, 'swing': 24.0, 'chair': 1257.0, 'scooter': 52.0, 'skiprope': 4232.0}}


In [71]:

# Cost per cubic meter per month for each state
cost_per_m3 = {
    'NY': 60,
    'ND': 10,
    'IL': 27,
    'TX': 19,
    'MO': 15,
    'CA': 40,
    'TN': 10,
    'PA': 30,
    'UT': 20,
    'WA': 29
}

# Initialize a dictionary to store the total sum for each state
total_sums = {}
# Initialize a dictionary to store the total cost for each state
total_costs = {}

# Calculate the total volume for each state
for state, items in volumes_perStates.items():
    total_sum = sum(items.values())
    total_sums[state] = total_sum

# Calculate the total cost for each state
for state, total_sum in total_sums.items():
    if state in cost_per_m3:
        total_costs[state] = total_sum * cost_per_m3[state]

total_cost_sum = sum(total_costs.values())
print(total_cost_sum)

print(warehousing_storageCost['DC'])
print(cost_per_m3)

KeyboardInterrupt: 